In [1]:
import requests
import json
import pandas as pd
from datetime import datetime
import statistics
from IPython.display import HTML, display
from wimbledon.vis.Visualise import DataHandlers

In [2]:
with open('github.token', 'r') as f:
    token = f.read().strip()

headers = {"Authorization": "Bearer " + token}


def run_query(query):  # A simple function to use requests.post to make the API call. Note the json= section.
    request = requests.post('https://api.github.com/graphql', json={'query': query}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(request.status_code, query))


GitHub emojis
=======

Project issues that members of the REG team have selected, "LAUGH" are the ones they would most like to be assigned to, "THUMBS_UP" they would also be happy with, "THUMBS_DOWN" they would not be happy with.

In [3]:
emojis = {'CONFUSED': '😕',
          'EYES': '👀',
          'HEART': '❤️',
          'HOORAY': '🎉',    
          'ROCKET': '🚀',
          'THUMBS_DOWN': '❌',
          'THUMBS_UP': '👍',
          'LAUGH': '✅'}

In [4]:
fc = DataHandlers.Forecast()  # get data from forecast

Enpoint: whoami | Time: 0.8866732619935647 
Enpoint: people/475627 | Time: 0.5444961390167009 

AUTHENTICATED USER:
Ed Chalstrey echalstrey@turing.ac.uk

CLIENTS
Enpoint: clients | Time: 0.6839039059996139 
PROJECTS
Enpoint: projects | Time: 0.8366309900011402 
ROLES
Enpoint: roles | Time: 0.906208628002787 
PEOPLE
Enpoint: people | Time: 1.0753536529955454 
PLACEHOLDERS
Enpoint: placeholders | Time: 1.4407947269792203 
MILESTONES
Enpoint: milestones | Time: 1.0463267840095796 
ASSIGNMENTS
Enpoint: assignments | Time: 2.1641590500075836 
DONE! (9.7s)


REG team
=====

We need a list of the team members full names from Forecast to compare with their names on GitHub. They match most of the time except when there is no GitHub name. Mapping below covers the rest of the team. We need to update this list if someone new is added to Forecast who doesn't have a matching GitHub name.

In [5]:
names = list(fc.people.full_name)
names.remove('Giovanni Colavizza')
names.remove('Miguel Morin')
names.remove('Mohammed Ali Al-Badri')
names

['Oliver Strickson',
 'Angus Williams',
 'David Beavan',
 'Evelina Gabasova',
 'James Geddes',
 'James Hetherington',
 'Louise Bowler',
 "Martin O'Reilly",
 'May Yong',
 'Nick Barlow',
 'Radka Jersakova',
 'Timothy Hobson',
 'Tomas Lazauskas',
 'Camila Rangel Smith',
 'Sarah Gibson',
 'James Robinson',
 'Eric Daub',
 'Jim Madge',
 'Amber Raza',
 'Kasra Hosseini',
 'Oscar Giles',
 'Joel Dearden',
 'Amaani Hoddoon',
 'Jack Roberts',
 'Oliver Forrest',
 'Gabriel Hanganu',
 'Ed Chalstrey',
 'Ashwini Venkatasubramaniam',
 'Roly Perera']

Find a team member's availability for a date range
---

In [6]:
def get_person_availability(peopledf, person, start_date, end_date):
    """Get the mean of a person's FTE proportion available for the start to end datetime objects"""
    if isinstance(person, str):
        try:
            person = fc.get_person_id(person)
        except:
            return 0.0
    peopledf = peopledf[(peopledf.index >= start_date) & (peopledf.index <= end_date)]
    try:
        availability_range = peopledf[person]
    except:
        return 0.0
    average_availability = statistics.mean(availability_range)
    return round(average_availability, 2)

In [7]:
peopledf = 1 - fc.people_totals.resample('MS').mean()  # pandas df for team members availability
start_date = datetime(2019, 10, 1)
end_date = datetime(2020, 10, 1)
print(get_person_availability(peopledf, "Ed Chalstrey", start_date, end_date))
print(get_person_availability(peopledf, 458589, start_date, end_date))

0.77
1.0


Projects should have a consistent resource required number over the months they aren't 0, but may be useful to look at the resource requirement over a specific date range:

In [8]:
def get_project_requirement(projectdf, project, start_date, end_date):
    """Get the mean of a project's FTE requirement for the start to end datetime objects"""
    if isinstance(project, str):
        try:
            project = fc.get_project_id(project)
        except:
            return 0.0
    projectdf = projectdf[(projectdf.index >= start_date) & (projectdf.index <= end_date)]
    try:
        requirement_range = projectdf[project]
    except:
        return 0.0
    average_requirement = statistics.mean(requirement_range)
    return round(average_requirement, 2)

In [9]:
projectdf = fc.project_resourcereq.resample('MS').mean()  # pandas df for team members availability
print(get_project_requirement(projectdf, "CHANCE", start_date, end_date))
print(get_project_requirement(projectdf, 2061048, start_date, end_date))

0.31
0.69


Create queries with GraphQL to get the emojis for each GitHub issue
===

https://developer.github.com/v4/guides/forming-calls/

In [10]:
query = """
{
  repository(owner:"alan-turing-institute", name:"Hut23") {
    issue(number:X) {
          number
          title
          url

          reactionGroups {
            content
            users(first:20) {
                edges {
                    node {
                        login
                        name
                    }
                }
            }
            }
    }
  }
}
"""

Check if a GitHub issue has emojis, based on the result of GraphQL query
----

In [11]:
def query_result_contains_emojis(query_result):
    for reaction in query_result['data']['repository']['issue']['reactionGroups']:
        if len(reaction['users']['edges']) > 0:
            return True
    return False

In [12]:
def get_preference_data(fc):
    """Get each team members preference emoji for all projects with a GitHub issue"""
    issues = fc.projects["GitHub"].dropna()  # Get list of GitHub issues for projects
    gid_mapping = {  # People without their full names on github.
     'myyong': 'May Yong',
     'nbarlowATI': 'Nick Barlow',
     'thobson88': 'Timothy Hobson',
     'miguelmorin': 'Miguel Morin',
     'OscartGiles': 'Oscar Giles',
     'AshwiniKV': 'Ashwini Venkatasubramaniam',
    }
    emoji_mapping = {'CONFUSED': '😕',
          'EYES': '👀',
          'HEART': '❤️',
          'HOORAY': '🎉',    
          'ROCKET': '🚀',
          'THUMBS_DOWN': '❌',
          'THUMBS_UP': '👍',
          'LAUGH': '✅'}
    names = list(fc.people.full_name)
    names.remove('Giovanni Colavizza')
    names.remove('Miguel Morin')
    names.remove('Mohammed Ali Al-Badri')
    preference_data = {
        "Person": names
    }
    for issue_num, project_id in zip(issues, issues.index):
        modified_query = query.replace("X", str(issue_num))
        result = run_query(modified_query)  # Execute the query
#         if query_result_contains_emojis(result)  # Could do this if we only want issues with emojis
        emojis = []
        for name in names: 
            emoji_name = None
            for reaction in result['data']['repository']['issue']['reactionGroups']:
                for edge in reaction['users']['edges']:
                    if edge['node']['name'] == name:
                        emoji_name = reaction['content']
                        if not emoji_name:
                            if gid_mapping[edge['node']['login']] == name:
                                emoji_name = reaction['content']
            if emoji_name:
                emoji = emoji_mapping[emoji_name]
            else:
                emoji = "❓"
            emojis.append(emoji)
        preference_data[fc.get_project_name(project_id)] = emojis
    return preference_data

In [13]:
preference_data = get_preference_data(fc)

In [14]:
preference_data_df = pd.DataFrame(preference_data).set_index('Person')
preference_data_df

Sargasso (Intel Sparse) DetectorChecker HSBC EDS  \
Person                                                                        
Oliver Strickson                                 ❓               ❓        ❓   
Angus Williams                                   ❓               ❓        ❓   
David Beavan                                     ❓               ❓        ❓   
Evelina Gabasova                                 ❓               ❓        ❓   
James Geddes                                     ❓               ❓        ❓   
James Hetherington                               ❓               ❓        ❓   
Louise Bowler                                    ❓               ❓        ❓   
Martin O'Reilly                                  ❓               ❓        ❓   
May Yong                                         ❓               ❓        ❓   
Nick Barlow                                      ❓               ❓        ❓   
Radka Jersakova                                  ❓               ❓        ❓   
Timothy Hobson                                   ❓               ❓        ❓   
Tomas Lazauskas                                  ❓               ❓        ❓   
Camila Rangel Smith                              ❓               ❓        ❓   
Sarah Gibson                                     ❓               ❓        ❓   
James Robinson                                   ❓               ❓        ❓   
Eric Daub                                        ❓               ❓        ❓   
Jim Madge                                        ❓               ❓        ❓   
Amber Raza                                       ❓               ❓        ❓   
Kasra Hosseini                                   ❓               ❓        ❓   
Oscar Giles                                      ❓               ❓        ❓   
Joel Dearden                                     ❓               ❓        ❓   
Amaani Hoddoon                                   ❓               ❓        ❓   
Jack Roberts                                     ❓               ❓        ❓   
Oliver Forrest                                   ❓               ❓        ❓   
Gabriel Hanganu                                  ❓               ❓        ❓   
Ed Chalstrey                                     ❓               ❓        ❓   
Ashwini Venkatasubramaniam                       ❓               ❓        ❓   
Roly Perera                                      ❓               ❓        ❓   

                           AI for City Planning Reproducible Research PDQ  \
Person                                                                      
Oliver Strickson                              ❓                     ❓   ❓   
Angus Williams                                ❓                     ❓   ❓   
David Beavan                                  ❓                     ❓   ❓   
Evelina Gabasova                              ❓                     ❓   ❓   
James Geddes                                  ❓                     ❓   ❓   
James Hetherington                            ❓                     ❓   ❓   
Louise Bowler                                 ❓                     ❓   ❓   
Martin O'Reilly                               ❓                     ❓   ❓   
May Yong                                      ❓                     ❓   ❓   
Nick Barlow                                   ❓                     ❓   ❓   
Radka Jersakova                               ❓                     ❓   ❓   
Timothy Hobson                                ❓                     ❓   ❓   
Tomas Lazauskas                               ❓                     ❓   ❓   
Camila Rangel Smith                           ❓                     ❓   ❓   
Sarah Gibson                                  ❓                     ❓   ❓   
James Robinson                                ❓                     ❓   ❓   
Eric Daub                                     ❓                     ❓   ❓   
Jim Madge                                     ❓                     ❓   ❓   
Amber Raza                           

In [15]:
preference_data_df["CHANCE"]["Ed Chalstrey"]

'✅'

Get the projects for which resources (team members) are required and display availability and preference for each team member who has with reacted with an emoji on the GitHub issue
----

In [20]:
def get_preferences(fc, preference_data_df, first_date=False, last_date=False, person=False, project=False, positive_only=False, emojis_only=False):
    resreqdf = fc.project_resourcereq.resample('MS').mean() # grouped by month and mean taken
    if person:
        names = [person]
    else:
        names = list(preference_data_df.index)
    data = {
        "Person": names
    }
    issues = fc.projects["GitHub"].dropna().values
    peopledf = 1 - fc.people_totals.resample('MS').mean()
    projectdf = fc.project_resourcereq.resample('MS').mean()  # pandas df for team members availability
    for project_id in resreqdf:  # some of these have no GitHub issue
        date_indices = resreqdf.index[resreqdf[project_id] != 0]
        if len(resreqdf.index[resreqdf[project_id] != 0]) > 0:  # if at least one month in the dataframe has a resource requirement of more than 0 FTE
            issue_num = fc.projects.loc[project_id, "GitHub"]
            if issue_num in issues:  # if this project has a GitHub issue
#                 start_date = fc.projects.loc[project_id, "start_date"]
#                 end_date = fc.projects.loc[project_id, "end_date"]
                first_resreq_date = date_indices[0].strftime("%Y-%m-%d")
                last_resreq_date = date_indices[-1].strftime("%Y-%m-%d")
                resreq = get_project_requirement(projectdf, project_id, first_resreq_date, last_resreq_date)
                project_title = fc.projects.loc[project_id, "name"]
                emoji_data = []
                for name in names:                 
                    person_availability = get_person_availability(peopledf, name, first_resreq_date, last_resreq_date)
                    percentage_availability = round((person_availability / resreq) * 100, 2)
                    emoji = preference_data_df[project_title][name]
                    if emojis_only:
                        emoji_data.append(emoji)
                    else:
                        emoji_data.append(emoji + " " + str(percentage_availability) + "% (" + str(person_availability) + ")")
                data[project_title] = emoji_data
    return pd.DataFrame(data).set_index('Person')

All projects with team members preferences and their average availability between the start of project resource requirement and project end date
----

In [22]:
emoji_table = get_preferences(fc, preference_data_df, emojis_only=True).to_html(show_dimensions=True)
emoji_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Living With Machines</th>\n      <th>AIDA-Lloyds</th>\n      <th>Probabilistic FEM</th>\n      <th>Safe Haven</th>\n      <th>Learning Machines</th>\n      <th>UQM^3</th>\n      <th>Auto DL</th>\n      <th>All in One Cancer imaging optimisation</th>\n      <th>Fusion modelling- IDEA</th>\n      <th>Uncertainty in Government Modelling</th>\n      <th>Interpretable AI for health</th>\n      <th>Micro-location analytics</th>\n      <th>Machine learning for cancer</th>\n      <th>Urban systems resilience</th>\n      <th>NCSC</th>\n      <th>Visualisation: Roy Ruddle TF project</th>\n      <th>SPF TPS 2019+</th>\n      <th>Logics for DS</th>\n      <th>Medevac Decision Support</th>\n      <th>NHS Medication Safety</th>\n      <th>UCLH Cancer Waiting Times</th>\n      <th>NHS Scotland Cancer Waiting Times</th>\n      <th>DFTG Commons scoping</th>\n      <th>DFTG Commons WP 3.4/3.

<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Living With Machines</th>\n      <th>AIDA-Lloyds</th>\n      <th>Probabilistic FEM</th>\n      <th>Safe Haven</th>\n      <th>Learning Machines</th>\n      <th>UQM^3</th>\n      <th>Auto DL</th>\n      <th>All in One Cancer imaging optimisation</th>\n      <th>Fusion modelling- IDEA</th>\n      <th>Uncertainty in Government Modelling</th>\n      <th>Interpretable AI for health</th>\n      <th>Micro-location analytics</th>\n      <th>Machine learning for cancer</th>\n      <th>Urban systems resilience</th>\n      <th>NCSC</th>\n      <th>Visualisation: Roy Ruddle TF project</th>\n      <th>SPF TPS 2019+</th>\n      <th>Logics for DS</th>\n      <th>Medevac Decision Support</th>\n      <th>NHS Medication Safety</th>\n      <th>UCLH Cancer Waiting Times</th>\n      <th>NHS Scotland Cancer Waiting Times</th>\n      <th>DFTG Commons scoping</th>\n      <th>DFTG Commons WP 3.4/3.5</th>\n      <th>CHANCE</th>\n      <th>Scaleable Monte Carlo</th>\n      <th>Wrattler 2019-2020</th>\n      <th>Census 2021 backbrief</th>\n      <th>Solar Panel Detection</th>\n      <th>NoCell - Phase 2</th>\n      <th>The Turing Way - Phase 2</th>\n      <th>Hate Speech</th>\n      <th>SAIL multimorbidity</th>\n    </tr>\n    <tr>\n      <th>Person</th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Oliver Strickson</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>Angus Williams</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>David Beavan</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>Evelina Gabasova</th>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n    </tr>\n    <tr>\n      <th>James Geddes</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>✅</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>James Hetherington</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>✅</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>Louise Bowler</th>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>✅</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n    </tr>\n    <tr>\n      <th>Martin O\'Reilly</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>May Yong</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>Nick Barlow</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>Radka Jersakova</th>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n    </tr>\n    <tr>\n      <th>Timothy Hobson</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>Tomas Lazauskas</th>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>✅</td>\n      <td>✅</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n    </tr>\n    <tr>\n      <th>Camila Rangel Smith</th>\n      <td>👍</td>\n      <td>✅</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>✅</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>✅</td>\n      <td>✅</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>✅</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n    </tr>\n    <tr>\n      <th>Sarah Gibson</th>\n      <td>✅</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>✅</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>James Robinson</th>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>✅</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>✅</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n    </tr>\n    <tr>\n      <th>Eric Daub</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n    </tr>\n    <tr>\n      <th>Jim Madge</th>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n    </tr>\n    <tr>\n      <th>Amber Raza</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>Kasra Hosseini</th>\n      <td>✅</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n    </tr>\n    <tr>\n      <th>Oscar Giles</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>Joel Dearden</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>Amaani Hoddoon</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>Jack Roberts</th>\n      <td>✅</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>✅</td>\n      <td>✅</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>👍</td>\n    </tr>\n    <tr>\n      <th>Oliver Forrest</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>Gabriel Hanganu</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>Ed Chalstrey</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>✅</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>✅</td>\n      <td>✅</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>✅</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>👍</td>\n    </tr>\n    <tr>\n      <th>Ashwini Venkatasubramaniam</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n    </tr>\n    <tr>\n      <th>Roly Perera</th>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>❓</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❓</td>\n      <td>👍</td>\n      <td>❌</td>\n      <td>❌</td>\n      <td>❌</td>\n    </tr>\n  </tbody>\n</table>\n<p>29 rows × 33 columns</p>

In [18]:
get_preferences(fc, preference_data_df)

Living With Machines      AIDA-Lloyds  \
Person                                                             
Oliver Strickson                ❓ 38.36% (0.61)     ❓ 0.0% (0.0)   
Angus Williams                     ❓ 0.0% (0.0)     ❓ 0.0% (0.0)   
David Beavan                    ❓ 25.79% (0.41)     ❓ 0.0% (0.0)   
Evelina Gabasova                👍 25.79% (0.41)     👍 0.0% (0.0)   
James Geddes                   ❓ -5.66% (-0.09)     ❓ 0.0% (0.0)   
James Hetherington                ❓ 6.29% (0.1)    ❓ 25.0% (0.1)   
Louise Bowler                    👍 50.31% (0.8)   👍 150.0% (0.6)   
Martin O'Reilly                 ❓ 11.32% (0.18)     ❓ 0.0% (0.0)   
May Yong                         ❓ 50.31% (0.8)    ❓ 50.0% (0.2)   
Nick Barlow                     ❓ 48.43% (0.77)     ❓ 0.0% (0.0)   
Radka Jersakova                 👍 55.97% (0.89)   ❓ 125.0% (0.5)   
Timothy Hobson                  ❓ 25.79% (0.41)     ❓ 0.0% (0.0)   
Tomas Lazauskas                 ❌ 46.54% (0.74)    ❌ 50.0% (0.2)   
Camila Rangel Smith              👍 52.2% (0.83)    ✅ 25.0% (0.1)   
Sarah Gibson                     ✅ 44.03% (0.7)     👍 0.0% (0.0)   
James Robinson                   👍 44.03% (0.7)    👍 2.5% (0.01)   
Eric Daub                        ❓ 44.03% (0.7)     ❓ 0.0% (0.0)   
Jim Madge                       ❌ 49.69% (0.79)   ❓ 125.0% (0.5)   
Amber Raza                       ❓ 62.89% (1.0)   ❓ 250.0% (1.0)   
Kasra Hosseini                  ✅ 45.91% (0.73)   👍 100.0% (0.4)   
Oscar Giles                     ❓ 57.23% (0.91)  ❓ 127.5% (0.51)   
Joel Dearden                       ❓ 0.0% (0.0)     ❓ 0.0% (0.0)   
Amaani Hoddoon                   ❓ 62.89% (1.0)   ❓ 250.0% (1.0)   
Jack Roberts                    ✅ 42.14% (0.67)   ❓ 125.0% (0.5)   
Oliver Forrest                   ❓ 62.89% (1.0)   ❓ 250.0% (1.0)   
Gabriel Hanganu                  ❓ 62.89% (1.0)   ❓ 250.0% (1.0)   
Ed Chalstrey                    ❓ 24.53% (0.39)   ❓ 250.0% (1.0)   
Ashwini Venkatasubramaniam      ❓ 25.79% (0.41)   ❓ 150.0% (0.6)   
Roly Perera                      ❓ 18.87% (0.3)   ❓ 250.0% (1.0)   

                           Probabilistic FEM       Safe Haven  \
Person                                                          
Oliver Strickson              ❓ 24.0% (0.12)   ❓ 8.33% (0.11)   
Angus Williams                  ❓ 0.0% (0.0)     ❓ 0.0% (0.0)   
David Beavan                  ❓ 24.0% (0.12)   ❓ 8.33% (0.11)   
Evelina Gabasova              ❌ 24.0% (0.12)   ❓ 8.33% (0.11)   
James Geddes                    ❓ 0.0% (0.0)     ❓ 0.0% (0.0)   
James Hetherington             ✅ 20.0% (0.1)   ❓ 6.82% (0.09)   
Louise Bowler                ❌ 112.0% (0.56)  ❓ 29.55% (0.39)   
Martin O'Reilly                 ❓ 0.0% (0.0)     ❓ 0.0% (0.0)   
May Yong                      ❓ 50.0% (0.25)  ❓ 13.64% (0.18)   
Nick Barlow                   ❓ 24.0% (0.12)   ❓ 8.33% (0.11)   
Radka Jersakova              ❌ 112.0% (0.56)  ❓ 29.55% (0.39)   
Timothy Hobson                ❓ 24.0% (0.12)   ❓ 8.33% (0.11)   
Tomas Lazauskas               ❌ 62.0% (0.31)  👍 24.24% (0.32)   
Camila Rangel Smith           ❌ 62.0% (0.31)  ❓ 18.94% (0.25)   
Sarah Gibson                  ❌ 24.0% (0.12)   ❓ 8.33% (0.11)   
James Robinson                ❌ 26.0% (0.13)   ❌ 8.33% (0.11)   
Eric Daub                     ❓ 24.0% (0.12)   ❓ 8.33% (0.11)   
Jim Madge                     👍 100.0% (0.5)  ❓ 27.27% (0.36)   
Amber Raza                    ❓ 200.0% (1.0)   ❓ 75.76% (1.0)   
Kasra Hosseini                ❌ 62.0% (0.31)  ❌ 16.67% (0.22)   
Oscar Giles                  ❓ 126.0% (0.63)  ❓ 40.91% (0.54)   
Joel Dearden                    ❓ 0.0% (0.0)     ❓ 0.0% (0.0)   
Amaani Hoddoon                ❓ 200.0% (1.0)   ❓ 75.76% (1.0)   
Jack Roberts                  👍 100.0% (0.5)  ❓ 27.27% (0.36)   
Oliver Forrest                ❓ 200.0% (1.0)   ❓ 75.76% (1.0)   
Gabriel Hanganu               ❓ 200.0% (1.0)   ❓ 75.76% (1.0)   
Ed Chalstrey                  ❓ 200.0% (1.0)  ❓ 51.52% (0.68)   
Ashwini Venkatasubramaniam   ❓ 13

In [19]:
get_preferences(fc, preference_data_df, person="Ed Chalstrey")

Living With Machines     AIDA-Lloyds Probabilistic FEM  \
Person                                                                
Ed Chalstrey      ❓ 24.53% (0.39)  ❓ 250.0% (1.0)    ❓ 200.0% (1.0)   

                   Safe Haven Learning Machines            UQM^3  \
Person                                                             
Ed Chalstrey  ❓ 51.52% (0.68)   ❓ 41.62% (0.77)  ❓ 186.0% (0.93)   

                     Auto DL All in One Cancer imaging optimisation  \
Person                                                                
Ed Chalstrey  ❌ 200.0% (1.0)                         ✅ 46.0% (0.46)   

             Fusion modelling- IDEA Uncertainty in Government Modelling  ...  \
Person                                                                   ...   
Ed Chalstrey         ❓ 92.0% (0.92)                      ❓ 200.0% (1.0)  ...   

             DFTG Commons WP 3.4/3.5           CHANCE Scaleable Monte Carlo  \
Person                                                                        
Ed Chalstrey          ❓ 8.25% (0.33)  ✅ 192.0% (0.96)          👍 0.0% (0.0)   

             Wrattler 2019-2020 Census 2021 backbrief Solar Panel Detection  \
Person                                                                        
Ed Chalstrey     ❓ 75.19% (1.0)       👍 344.83% (1.0)        ❓ 200.0% (1.0)   

             NoCell - Phase 2 The Turing Way - Phase 2     Hate Speech  \
Person                                                                   
Ed Chalstrey  ❌ 134.0% (0.67)           ❌ 61.0% (0.61)  ❌ 200.0% (1.0)   

             SAIL multimorbidity  
Person                            
Ed Chalstrey        👍 0.0% (0.0)  

[1 rows x 33 columns]